# Machine learning applied to 0D reactors with multiple dt prediction

In [ ]:
use_colab = False

## Google colab preparation

These lines are here to enable Colab running of the tools. We need to perform a git clone in order to have access to python scripts. This needs to be done at each runtime as the clone is lost. 

In [ ]:
import os

if use_colab:
    !git clone -b cost_course_exercices https://github.com/cmehl/ML_chem.git
    
    !pip install cantera

    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Create a folder in the root directory
    if not os.path.isdir("/content/drive/MyDrive/ML_chem_data"):
        !mkdir -p "/content/drive/MyDrive/ML_chem_data"
    else:
        print("Folder /content/drive/MyDrive/ML_chem_data already exists")

## Imports and options

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import time
import json
import joblib
import numpy as np
import pandas as pd

import cantera as ct

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme("notebook")

if use_colab:
    from ML_chem.chem_ai.cantera_runs import compute_nn_cantera_0D_homo
    from ML_chem.chem_ai.utils import get_molar_mass_atomic_matrix
    from ML_chem.chem_ai.utils import StandardScaler
else:
    from chem_ai.cantera_runs import compute_nn_cantera_0D_homo
    from chem_ai.utils import get_molar_mass_atomic_matrix
    from chem_ai.utils import StandardScaler

We set the default pytorch precision to double. It slows down a little bit the training but it is the usual standard for CFD reacting flows applications.

In [ ]:
torch.set_default_dtype(torch.float64)

We identify the device (CPU or GPU) available on the machine. This will be used by pytorch to identify the device on which to train and use the model:

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('Running on the GPU')
else:
  device = torch.device('cpu')
  print('Running on the CPU')

## Preliminary

We define the folder including the desired database:

In [ ]:
if use_colab:
    folder = "/content/drive/MyDrive/ML_chem_data/case_0D_test_multidt"
else:
    folder = "./case_0D_test_multidt" 

We load the parameters stored in the json file of the dabatase folder:

In [ ]:
with open(os.path.join(folder, "dtb_params.json"), "r") as file:
    dtb_params = json.load(file)

fuel = dtb_params["fuel"]
mech_file = dtb_params["mech_file"]
log_transform = dtb_params["log_transform"]
threshold = dtb_params["threshold"]
p = dtb_params["p"]
dt = dtb_params["dt"]

We load the scaler:

In [ ]:
Xscaler = joblib.load(os.path.join(folder, "processed_database", "Xscaler.pkl"))

We load the training and validation databases:

In [ ]:
X_train = np.load(os.path.join(folder, "processed_database", "X_train.npy"))
X_val = np.load(os.path.join(folder, "processed_database", "X_val.npy"))
Y_train = np.load(os.path.join(folder, "processed_database", "Y_train.npy"))
Y_val = np.load(os.path.join(folder, "processed_database", "Y_val.npy"))

dt_array_train = np.load(os.path.join(folder, "dt_array_train.npy"))
dt_array_val = np.load(os.path.join(folder, "dt_array_val.npy"))

Number of input and output dimensions, and number of dt values:

In [ ]:
n_s_train = X_train.shape[0]
n_s_val = X_val.shape[0]

n_in = X_train.shape[1]
n_out = Y_train.shape[1]
nb_dt = Y_train.shape[2]

In [ ]:
gas = ct.Solution(mech_file)
A_element = get_molar_mass_atomic_matrix(gas.species_names, fuel, True)
print(A_element)

## Method 1: dt input method

In this first method, which we could qualify as brute force, we add *dt* as an input of the network. 

We first need to prepare datasets so that we have the list of input and corresponding outputs.

First we tackle inputs:

In [ ]:
# TRAINING
X_train_1 = np.empty((nb_dt*n_s_train,n_in+1))

for i_dt in range(nb_dt):
    X_train_1[i_dt::nb_dt,:-1] = X_train

X_train_1[:,-1] = dt_array_train.flatten()


# VALIDATION
X_val_1 = np.empty((nb_dt*n_s_val,n_in+1))

for i_dt in range(nb_dt):
    X_val_1[i_dt::nb_dt,:-1] = X_val

X_val_1[:,-1] = dt_array_val.flatten()


Then we focus on outputs:

In [ ]:
# TRAINING
Y_train_1 = np.empty((nb_dt*n_s_train,n_out))

for i_dt in range(nb_dt):
    Y_train_1[i_dt::nb_dt,:] = Y_train[:,:,i_dt]


# VALIDATION
Y_val_1 = np.empty((nb_dt*n_s_val,n_out))

for i_dt in range(nb_dt):
    Y_val_1[i_dt::nb_dt,:] = Y_val[:,:,i_dt]

In [ ]:
X_train_1 = torch.tensor(X_train_1, dtype=torch.float64)
Y_train_1 = torch.tensor(Y_train_1, dtype=torch.float64)
X_val_1 = torch.tensor(X_val_1, dtype=torch.float64)
Y_val_1 = torch.tensor(Y_val_1, dtype=torch.float64)

In [ ]:
Xscaler_mean = torch.from_numpy(Xscaler.mean)
Xscaler_std = torch.from_numpy(Xscaler.std)

In [ ]:
A_element = torch.tensor(A_element, dtype=torch.float64)

In [ ]:
X_train_1 = X_train_1.to(device)
Y_train_1 = Y_train_1.to(device)
X_val_1 = X_val_1.to(device)
Y_val_1 = Y_val_1.to(device)

Xscaler_mean = Xscaler_mean.to(device)
Xscaler_std = Xscaler_std.to(device)

A_element = A_element.to(device)

We now can generate the model. In this work, we will consider a simple Multi Layer Perceptron (MLP). We generate the model using Pytorch:

In [ ]:
class ChemNN_multi(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(n_in + 1, 100)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(100, 100)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(100, n_out)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.output(x)
        return x

The model is then instantiated and transferred to the GPU if present:

In [ ]:
model = ChemNN_multi()
print(model)

#Put model on GPU
model = model.to(device)

In [ ]:
n_epochs = 300
batch_size = 256

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def main_training_loop(X_train, X_val, Y_train, Y_val, loss_fn, optimizer, n_epochs, model, log_transform):

    # Array to store the loss and validation loss
    loss_list = np.empty(n_epochs)
    val_loss_list = np.empty(n_epochs//10)

    # Array to store sum of mass fractions: mean, min and max
    stats_sum_yk = np.empty((n_epochs//10,3))

    # Array to store elements conservation: mean, min and max
    stats_A_elements = np.empty((n_epochs//10,4,3))

    epochs = np.arange(n_epochs)
    epochs_small = epochs[::10]

    for epoch in range(n_epochs):

        # Training parameters
        for i in range(0, len(X_train), batch_size):

            Xbatch = X_train[i:i+batch_size]
            y_pred = model(Xbatch)
            ybatch = Y_train[i:i+batch_size]
            loss = loss_fn(y_pred, ybatch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_list[epoch] = loss

        # Computing validation loss and mass conservation metric (only every 10 epochs as it is expensive)
        if epoch%10==0:
            model.eval()  # evaluation mode
            with torch.no_grad():

                # VALIDATION LOSS
                y_val_pred = model(X_val)
                val_loss = loss_fn(y_val_pred, Y_val)

                # SUM OF MASS FRACTION
                #Inverse scale done by hand to stay with Torch arrays
                yk = Xscaler_mean[1:] + (Xscaler_std[1:] + 1e-7)*y_val_pred
                if log_transform:
                    yk = torch.exp(yk)
                sum_yk = yk.sum(axis=1)
                sum_yk = sum_yk.detach().cpu().numpy()
                stats_sum_yk[epoch//10,0] = sum_yk.mean() 
                stats_sum_yk[epoch//10,1] = sum_yk.min()
                stats_sum_yk[epoch//10,2] = sum_yk.max()

                # ELEMENTS CONSERVATION
                yval_in = Xscaler_mean[1:] + (Xscaler_std[1:] + 1e-7)*X_val[:,1:-1]
                if log_transform:
                    yval_in = torch.exp(yval_in)
                ye_in = torch.matmul(A_element, torch.transpose(yval_in, 0, 1))
                ye_out = torch.matmul(A_element, torch.transpose(yk, 0, 1))
                delta_ye = (ye_out - ye_in)/(ye_in+1e-10)
                delta_ye = delta_ye.detach().cpu().numpy()
                stats_A_elements[epoch//10, :, 0] = delta_ye.mean(axis=1)
                stats_A_elements[epoch//10, :, 1] = delta_ye.min(axis=1)
                stats_A_elements[epoch//10, :, 2] = delta_ye.max(axis=1)

            model.train()   # Back to training mode
            val_loss_list[epoch//10] = val_loss

        print(f"Finished epoch {epoch}")
        print(f"    >> Loss: {loss}")
        if epoch%10==0:
            print(f"    >> Validation loss: {val_loss}")

    return epochs, epochs_small, loss_list, val_loss_list, stats_sum_yk, stats_A_elements

In [ ]:
start_time = time.perf_counter()
epochs, epochs_small, loss_list, val_loss_list, stats_sum_yk, stats_A_elements = main_training_loop(X_train_1, X_val_1, Y_train_1, Y_val_1, loss_fn, optimizer, n_epochs, model, log_transform)
end_time = time.perf_counter()
print(f" TRAINING DURATION: {end_time-start_time} s")

We can define a function to analyze the training. We plot:

+ The training and validation losses

+ The evolution of $\sum_{k=1}^{N_S} Y_k$ (mean, min and max).

+ The elements conservation by plotting $100\times\delta Y_e$ for each element (C, H, O and N). The factor $100$ enables to get an error in \%.

In [ ]:
def plot_losses_conservation(epochs, epochs_small, loss_list, val_loss_list, stats_sum_yk, stats_A_elements):

    # LOSSES
    fig, ax = plt.subplots()

    ax.plot(epochs, loss_list, color="k", label="Training")
    ax.plot(epochs_small, val_loss_list, color="r", label = "Validation")

    ax.set_yscale('log')

    ax.legend()

    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")

    # MASS CONSERVATION
    fig, ax = plt.subplots()

    ax.plot(epochs_small, stats_sum_yk[:,0], color="k")
    ax.plot(epochs_small, stats_sum_yk[:,1], color="k", ls="--")
    ax.plot(epochs_small, stats_sum_yk[:,2], color="k", ls="--")

    ax.set_xlabel("Epoch")
    ax.set_ylabel(r"$\sum_k \ Y_k$")

    # ELEMENTS CONSERVATION
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)

    # C
    ax1.plot(epochs_small, 100*stats_A_elements[:,0,0], color="k")
    ax1.plot(epochs_small, 100*stats_A_elements[:,0,1], color="k", ls="--")
    ax1.plot(epochs_small, 100*stats_A_elements[:,0,2], color="k", ls="--")

    ax1.set_xlabel("Epoch")
    ax1.set_ylabel(r"$\Delta Y_C$ $(\%$)")

    # H
    ax2.plot(epochs_small, 100*stats_A_elements[:,1,0], color="k")
    ax2.plot(epochs_small, 100*stats_A_elements[:,1,1], color="k", ls="--")
    ax2.plot(epochs_small, 100*stats_A_elements[:,1,2], color="k", ls="--")

    ax2.set_xlabel("Epoch")
    ax2.set_ylabel(r"$\Delta Y_H$ $(\%)$")

    # O
    ax3.plot(epochs_small, 100*stats_A_elements[:,2,0], color="k")
    ax3.plot(epochs_small, 100*stats_A_elements[:,2,1], color="k", ls="--")
    ax3.plot(epochs_small, 100*stats_A_elements[:,2,2], color="k", ls="--")

    ax3.set_xlabel("Epoch")
    ax3.set_ylabel(r"$\Delta Y_O$ $(\%)$")

    # N
    ax4.plot(epochs_small, 100*stats_A_elements[:,3,0], color="k")
    ax4.plot(epochs_small, 100*stats_A_elements[:,3,1], color="k", ls="--")
    ax4.plot(epochs_small, 100*stats_A_elements[:,3,2], color="k", ls="--")

    ax4.set_xlabel("Epoch")
    ax4.set_ylabel(r"$\Delta Y_N$ $(\%)$")

    fig.tight_layout()

In [ ]:
plot_losses_conservation(epochs, epochs_small, loss_list, val_loss_list, stats_sum_yk, stats_A_elements)